In [100]:
!pip install ndjson

In [101]:
import ndjson
import re
from collections import defaultdict
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
import pandas as pd

In [102]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [103]:
term_doc_index = {}
with open('/content/drive/MyDrive/MTech/sem1/IR/merged.txt') as f:
  next(f)
  for line in f:
    word, *docs = line.strip().split(' ')
    term_doc_index[word] = tuple(docs)

In [104]:
doc_id_to_title = {}
shelf_to_title = defaultdict(list)
i=0

f = open('/content/cran_experiment.ndjson')
data_file = ndjson.reader(f)
for doc in data_file:
    title = doc["title"], 
    author = doc["author"], 
    id=doc["id"], 
    meta = doc["meta"]
    data = doc["data"]
    i += 1
    doc_id_to_title[id[0]] = title

In [105]:
def remove_symbols(line):
    return re.sub('[^A-Za-z0-9\s]+', '', line).lower()
def preprocess_term(term, stem=1):
    term = term.lower()
    #condition of with/without stemming
    return remove_symbols(word) if not stem else remove_symbols(porter.stem(term))
preprocess_term('satisfaction')

'satisfact'

In [106]:
#query method for quering documents
def query(include=[], dont_include=[]):
  all_doc_ids = set()
  for term in include:
    term = preprocess_term(term)
    docs = term_doc_index.get(term, [])
    #if there is atleast one doc in all_doc_ids
    if all_doc_ids:
        #if the doc has ALL THE WORDS
        all_doc_ids.intersection_update(set(docs))
    #if doc has no word
    else:
        all_doc_ids.update(set(docs))
    #if all_doc_ids is empty, stop executing this method
    if not all_doc_ids:
      break
#   all_doc_ids = [int(i.replace('(','').replace(',)','')) for i in all_doc_ids]  
#   print("ALL DOCS:\n", type(list(all_doc_ids)[0]))
  complement_docs = set()
  for term in dont_include:
    term = preprocess_term(term)
    docs = term_doc_index.get(term, [])
    complement_docs.update(set(docs))
    #if there is atleast one doc in complement_docs
    if complement_docs:
        # removing the docs which have all terms which we dont want
        set(all_doc_ids).difference_update(set(docs))
        doc_titles = [doc_id_to_title[i] for i in all_doc_ids]
    #returning the doc titles as per the query
    # return all_doc_ids
    return doc_titles

query(include=['experimental'], dont_include=['investigation'])

[('breathing vibrations of a circular shell with an internal\nliquid .',),
 ('a wind-tunnel test technique for measuring the dynamic rotary\nstability derivatives at subsonic and supersonic speeds .',),
 ('transition form laminar to turbulent shear flow .',),
 ('stagnation point of a blunt body in hypersonic flow .',),
 ('a study of the simulation of flow with free stream mach number 1 in a\nchoked wind tunnel .',),
 ('jet effects on base pressure of conical afterbodies\nat mach 1. 91 and 3. 12 .',),
 ('flow past slender blunt bodies - a review and extension .',),
 ('the hovercraft - a new concept in maritime transport .',),
 ('the drag of elongated bodies over a wide reynolds number\nrange .',),
 ('laminar heat transfer around blunt bodies in dissociated\nair .',),
 ('thermodynamic coupling in boundary layers .',),
 ('investigation to determine effects of center of gravity location on the\ntransonic flutter characteristics of a 45degree sweptback wing .',),
 ('the flexural vibrations 

In [107]:
#precision, recall
# computing expected output
# qrel = []
# with open('/content/cranqrel') as f:
#     for line in f:
#         qrel.append(line.split())

# for q in range(len(qrel)):
#     if int(qrel[q][2]) < 4:
#         qrel[q][2] = '1'
#     else:
#         qrel[q][2] = '0'

# qrel